In [1]:
import tensorflow as tf
import numpy as np

In [2]:
char_arr = ["a", "b", "c", "d", "e", "f", "g", 
            "h", "i", "j", "k", "l", "m", "n", 
            "o", "p", "q", "r", "s", "t", "u", 
            "v", "w", "x", "y", "z"]

num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

In [3]:
seq_data = ["word", "wood", "deep", "dive", "cold", 
            "cool", "load", "love", "kiss", "kind"]

In [7]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        input_ = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        input_batch.append(np.eye(dic_len)[input_])
        target_batch.append(target)
        
    return input_batch, target_batch

In [5]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30

n_step = 3
n_input = n_class = dic_len

X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], 
                       feed_dict={X: input_batch, Y: target_batch})
    
    print("Epoch:", "{:04d}".format(epoch+1), 
          "Cost:", "{:.6f}".format(loss))
    
print("Optimization completed!")

Epoch: 0001 Cost: 4.105891
Epoch: 0002 Cost: 3.192192
Epoch: 0003 Cost: 1.706254
Epoch: 0004 Cost: 1.412473
Epoch: 0005 Cost: 0.910616
Epoch: 0006 Cost: 0.455543
Epoch: 0007 Cost: 0.804012
Epoch: 0008 Cost: 0.399443
Epoch: 0009 Cost: 0.407940
Epoch: 0010 Cost: 0.269022
Epoch: 0011 Cost: 0.190012
Epoch: 0012 Cost: 0.430488
Epoch: 0013 Cost: 0.129308
Epoch: 0014 Cost: 0.153969
Epoch: 0015 Cost: 0.095906
Epoch: 0016 Cost: 0.296049
Epoch: 0017 Cost: 0.070573
Epoch: 0018 Cost: 0.411647
Epoch: 0019 Cost: 0.061635
Epoch: 0020 Cost: 0.013756
Epoch: 0021 Cost: 0.423486
Epoch: 0022 Cost: 0.029293
Epoch: 0023 Cost: 0.078512
Epoch: 0024 Cost: 0.083243
Epoch: 0025 Cost: 0.015184
Epoch: 0026 Cost: 0.080323
Epoch: 0027 Cost: 0.057226
Epoch: 0028 Cost: 0.015416
Epoch: 0029 Cost: 0.003906
Epoch: 0030 Cost: 0.009726
Optimization completed!


In [12]:
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy], 
                                 feed_dict={X: input_batch, Y: target_batch})

predict_words = []

for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)
    
print("\n=== prediction ===")
print("input:", [w[:3] + ' ' for w in seq_data])
print("prediction:", predict_words)
print("accuracy:", accuracy_val)


=== prediction ===
input: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
prediction: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
accuracy: 1.0
